In [1]:
import pandas as pd
import numpy as np
import random

randomlist = []
df_demand = {}
random.seed(12) # Use the same random seed
for i in range(0,50):   # 20 depots
    n = random.randint(100,250) # Depot Capacity
    randomlist.append(n)

for i in range(50):
    df_demand[i] = randomlist[i]

sum = 0
for (k,v) in df_demand.items():
    sum = sum + v
    
df_supply = {}
for i in range(sum):
    df_supply[i] = 1.0
    
np.random.seed(12) # Use the same random seed
df_dkm = pd.DataFrame(np.random.uniform(0.5,40, size=(sum,50))).astype(float)

random.seed(12) # Use the same random seed
age_b = []
for i in range(sum):
    n = random.randint(1,3) #Only to generate integer values inclusive of 1 & 3
    age_b.append(n)
    
age = {}
for i in range(len(age_b)):
    age[i] = age_b[i] 
    
kpl = []
co2 = []
random.seed(12) # Use the same random seed
for (k,v) in age.items():
    if (v==1):
        n = random.uniform(5.1,6)
        m = round(random.uniform(515, 524), 2)
        kpl.append(n)
        co2.append(m)
    if (v==2):
        n = random.uniform(4.1,5)
        m = round(random.uniform(525, 534), 2)
        kpl.append(n)
        co2.append(m)
    if (v==3):
        n = random.uniform(3.1,4)
        m = round(random.uniform(535, 540), 2)
        kpl.append(n)
        co2.append(m)
        
fc_kpl = {}
for i in range(sum):
    fc_kpl[i] =(111/ kpl[i])

co2_c = {}
for i in range(sum):
    co2_c[i] =(0.00118642*co2[i])
    
random.seed(12) # Use the same random seed
doc = {}
for i in range(0,50):
    n = random.uniform(50,100)
    doc[i] = n
    
df_tdkom = pd.DataFrame(np.zeros(df_dkm.shape))
for i in range(len(df_dkm.columns)):
    for j in range(len(df_dkm.index)):
        df_tdkom[i][j] = df_dkm[i][j]*fc_kpl[j] + df_dkm[i][j]*co2_c[j] + doc[i]
#df_tdkom
df_cost = df_tdkom.copy(deep=True)

In [2]:
# Ranking each depot depending on value of DK
df_rank= (df_tdkom.rank(1, ascending=True, method='first')).astype(int)     

df_depot = df_demand.copy()

R = 1
df_result = pd.DataFrame(np.zeros(df_tdkom.shape))

In [3]:
while(R!=len(df_depot)+1): 
    df_rankr = (df_rank == R)*1.0

    lst = []
    for m in df_rankr.columns:
        if (df_rankr.sum(axis=0)[m] <= df_demand[m]):
            for j in df_rankr.index:
                df_result.loc[j,m] = df_rankr.loc[j,m] 

    for m in df_rankr.columns:     
        if (df_rankr.sum(axis=0)[m] > df_demand[m]):
                for j in df_rankr.index:
                    if (df_rank.loc[j,m] == R):
                        lst.append((j,m))

                new = []
                for (k,v) in lst:
                    if (v == m):
                        new.append(df_tdkom.loc[k,v])      

                new.sort()
                n = int(df_demand[m])
                #print(n)
                newn = new[0:n]

                for j in df_rankr.index:
                    for i in range(len(newn)):
                        if (newn[i] == df_tdkom.loc[j,m]): 
                            df_result.loc[j,m] = df_rankr.loc[j,m]  

    for m in df_rankr.columns:
        df_demand[m] = df_depot[m] - df_result.sum(axis=0)[m] 

    for m in df_rankr.columns: 
        if (df_demand[m] == 0):
                df_rank = df_rank.drop(m, axis=1)
                del df_demand[m]

    for k in df_rankr.index:
        df_supply[k] = df_supply[k] - df_result.sum(axis=1)[k]

    for i in df_rankr.index:
        if (df_supply[i] == 0):
            df_rank = df_rank.drop(i, axis = 0)
            del df_supply[i]

    R = R + 1

In [4]:
df_result

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8843,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8844,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8845,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
8846,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df_final = df_cost.mul(df_result, axis = 0)
df_final.to_numpy().sum()

1059214.0049023556